In [4]:
spark.stop() #안하면 메모리에 계속 있음.

In [5]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("241204_01_RDD_API")
spark = SparkContext(conf = conf).getOrCreate()
spark

<SparkContext master=local appName=241204_01_RDD_API>

In [7]:
foods = spark.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"])
foods

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [8]:
foods.collect() #전부 메모리에 올림

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [9]:
foods.countByValue() #값을 기준으로 카운트

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [10]:
foods.take(3) #상위 3개만, collect에 비해 효율적인편

['짜장면', '마라탕', '짬뽕']

In [11]:
foods.first() #상위 1개만

'짜장면'

In [12]:
foods.count() #RDD의 개수 = 12개를 구해올것

12

In [14]:
fd_dist = foods.distinct()
fd_dist

PythonRDD[14] at RDD at PythonRDD.scala:53

In [15]:
fd_dist.collect() #action연산 : 결과를 직접 반환하는 연산을 의미

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [17]:
#워커노드에서 실행하는 기능
foods.foreach(lambda x : print(x))

짜장면
마라탕
짬뽕
떡볶이
쌀국수
짬뽕
짜장면
짜장면
짜장면
라면
우동
라면


In [ ]:
# narrow operation 1:1연산!
# filter(), map(), flatMap(), sample(), union()

In [18]:
sample_rdd = spark.parallelize([1,2,3,4,5])

In [19]:
sample_rdd2 = sample_rdd.map(lambda x: x+2)
sample_rdd2.collect() #2씩 더해져서 출력

[3, 4, 5, 6, 7]

In [20]:
# 영화제목 리스트 생성(예시)
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [21]:
moviesRDD = spark.parallelize(movies)

In [23]:
mapMovies = moviesRDD.map(lambda x: x.split(" ")) #공백을 기준으로 쪼개기
mapMovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [24]:
flatmapMovies = moviesRDD.flatMap(lambda x: x.split(" ")) #공백을 기준으로 쪼개기
flatmapMovies.collect() #map과의 차이점 확인

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [27]:
filteredMovies = flatmapMovies.filter(lambda x: x != "매트릭스")
filteredMovies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [28]:
# 집합연산
num1 = spark.parallelize([1,2,3,4,5])
num2 = spark.parallelize([4,5,6,7,8,9,10])

In [32]:
# intersection
num1.intersection(num2).collect()

[4, 5]

In [35]:
num1.subtract(num2).collect()

[2, 1, 3]

In [36]:
num2.subtract(num1).collect()

[6, 8, 10, 7, 9]

In [42]:
numlist = num1.union(num2)
numlist.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [37]:
# sample  : 데이터의 일부를 샘플링해서 추출
# sample(withReplacement, fraction, seed))

# withReplacement : 비복원, 복원 추출
# fraction : 기대값
# seed : 난수 추출을 위한 시드값

In [58]:
numlist.sample(True, 0.5).collect() #시드가 없어 출력할때마가 다름

[1, 1, 4, 4, 7, 7, 10]

In [59]:
numlist.sample(True, 0.5, seed=42).collect() #시드가 있어 항상 같은 값 출력

[5, 6, 6]

In [60]:
# WIDE TRANSFORMATION
# groupby reduce
foods.collect() #위에서 썼던 것 가져오기 

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [63]:
foodsGroup = foods.groupBy(lambda x: x[0]) #c첫번째 글자로 묶기
res = foodsGroup.collect()

In [67]:
for (k,v) in res: print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']


In [68]:
spark.stop() #안하면 메모리에 계속 있음.